# System of Linear Equations Solver
_By Dhruv Jain_

### **Objective: Solve a fully determined system of linear equations**

In [3]:
# Key libraries: Numpy(for mathematical procedures) and matplotlib(to create plots)
import numpy as np
import matplotlib.pyplot as plt 
import copy

## Gaussian Elimination with back subsitution without pivoting and with scaled partial pivoting

In [4]:
def ge_lin_equ(a,b,opt):
    """Dhruv Jain, 19 Sept, 2021
    Obj: To solve system of linear equations using Gaussian Elimination with No Pivoting(opt = 1) and Scaled Partial Pivoting (opt = 2)
    Args:
        a: (n x n) matrix, Coefficient Matrix
        b: (n x 1) vector, Right-hand side of the systems of equations
        opt: integer, 1 => GE with No Pivoting, 2 => GE with Scaled Partial Pivoting
    Returns:
        x: (n x 1) vector, computed solution of the system  
    """
    
    n = len(b) # Number of columns = Number of rows = length of b vector
    r = np.arange(n) # r = [1, 2, .... n-1, n] to track the location of equation 
    a_orig = a
    a = np.hstack((a, np.transpose(np.array([b])))) # Augmented matric [A | b]
    
    if opt == 1: # GE with No Pivoting
        for i in range(n):
            for j in range(i+1,n):
                if a[i,i] != 0: # Check if leading column element is not 0, if 0 then swap with row of nonzero leading element
                    lam = a[j,i]/a[i,i] # Common factor to make subsequent column elements 0
                    a[j,:] = a[j,:] - lam*a[i,:] # ERO3, so that all subsequent columen elements = 0
                else: # If leading column element is 0 then we use ERO1, r_i <-> r_j
                    k = j
                    while a[k,i] == 0 and k < n-1: # Check till non-zero element in the leading column found
                        k = k + 1

                    if i < n-1 and k == n-1 and a[k,i] == 0: # If all subsequent columns are zero then move to next column
                        j = n
                    else: # If a non-zero element found then interchange rows and track interchange using r
                        temp_a = copy.copy(a[i,:])
                        a[i,:] = copy.copy(a[k,:])
                        a[k,:] = copy.copy(temp_a)

                        temp = copy.copy(r[i])
                        r[i] = copy.copy(k)
                        r[k] = copy.copy(temp)
                        j = j - 1

    elif opt == 2: # GE with Scaled Partial Pivoting
        s = np.zeros(n)
        M = np.zeros(n)
        for i in range(n-1):
            for k in range(i,n):
                    origpos = r[k]
                    s[k] = max(np.abs(a_orig[origpos])) # Calculation of scale vector using elements of the original A
                    M[k] = np.abs(a[k,i])/s[k] # Calculation of scaled coefficients, to find pivot element
            
            # Interchange rows with pivot element calculated using scaled coefficients 
            posmax = np.argmax(M[i:])
            temp_a = copy.copy(a[i,:])
            a[i,:] = copy.copy(a[i+posmax,:])
            a[i+posmax,:] = copy.copy(temp_a)

            # Track the row change 
            temp = copy.copy(r[i])
            r[i] = copy.copy(i+posmax)
            r[i+posmax] = copy.copy(temp)

        # ERO3, to elimiate subsequent leading terms 
            for j in range(i+1,n):
                if a[i,i] != 0: # Check if leading column element is not 0, if 0 then swap with row of nonzero leading element
                    lam = a[j,i]/a[i,i] # Common factor to make subsequent column elements 0
                    a[j,:] = a[j,:] - lam*a[i,:]

    # Back subsitution to obtain x, solution vector from the above simplified upper traingle augmented matrix
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        if a[i,i] != 0:
            x[i]=(a[i,-1]-a[i,i+1:n].dot(x[i+1:n]))/a[i,i] 
        else:
            x[i] = 0
    
    return x    

## Complex Gradient Method (Minimization method)

In [21]:
# Conjugate Gradient Method:
def cgm(A,b,x0, tol, tol_div, Nmax):
    """Dhruv Jain, 4 Oct, 2021, MSAAE Purdue
    Obj: Use Complex Gradient Method (Minimization method) to iteratively solve for a solution of linear system of equations of the form Ax = b, x is the solution 
    Args:
        A: (n x n) matrix, Coefficeient matrix
        b: (n x 1) vector, right-hand side vector
        x0: (n x 1) vecotr, inital guess for x*
        tol: ||x_n+1 - x_n||infi, step convergence tolerance 
        tol_div: ||x_n+1 - x_n||infi, step Divergence tolerance
        Nmax: max allowable iterations for the iterative solver
    Output:
        xfinal: (n_iter x n), contains solved x for each iteration, last row is the converged solution
        iterations: # iterations to converge  
    """
    r = np.matmul(A,x0) - b # Initial calculation of the residul vector
    d = -r # Setup search direction vector
    x = np.zeros((Nmax,len(b))) # Create x to save solution calculated after each iteration
    x[0,:] = x0 # Assign 1st row as the Initial Guess
    
    # Goal is to find minima of f = 1/2x.T*A*x - b.T*x + c
    for i in range(Nmax-1): # Run loop for set maximum iterations
        lam = np.matmul(-d.T, r)/np.matmul(np.matmul(d.T,A),d) # Calculate step size
        x[i+1,:] = x[i,:] + lam*d # Update solution using calculated step size and search direction
        r = np.matmul(A,x[i+1,:]) - b # Update residual vector using new calculated solution
        
        if max(abs(x[i+1,:] - x[i,:])) < tol: # Check if solution converges below a set tolerance using the l-infinity norm
            xfinal = x[0:i+2,:] # Save solutions for all the iterations 
            break # Exit out of the loop
        elif max(abs(x[i+1,:] - x[i,:])) > tol_div: # Check if solution diverges beyond a set tolerance, if does then stop the loop and rerun with updated arguments
            print('Solution is diverging \n')
            xfinal = 0
            break # Exit out of the loop
        # If solution has not converged/diverged beyond set tolerance
        alpha = np.matmul(np.matmul(r.T,A),d)/np.matmul(np.matmul(d.T,A),d) # Calculate an appropriate scaling scalar
        d = -r + alpha*d # Update search direction using the updated scalar alpha
    
    iterations = i+1
    return xfinal, iterations

## Gauss-Seidel Method (Splitting Method)

In [29]:
# Gauss-Seidel Method:
def gsm(A,b,x0, tol, tol_div, Nmax):
    """Dhruv Jain, 4 Oct, 2021, MSAAE Purdue
    Obj: Use Gauss-Seidel Method (Splitting Method) to iteratively solve for a solution of linear system of equations of the form Ax = b, x is the solution 
    Args:
        A: (n x n) matrix, Coefficeient matrix
        b: (n x 1) vector, right-hand side vector
        x0: (n x 1) vecotr, inital guess for x*
        tol: ||x_n+1 - x_n||infi, step convergence tolerance 
        tol_div: ||x_n+1 - x_n||infi, step Divergence tolerance 
        Nmax: max allowable iterations for the iterative solver
    Output:
        xfinal: (niter x n), contains solved x for each iteration, last row is the converged solution
        iterations: # iterations to converge  
    """
    x = np.zeros((Nmax,len(b))) # Create x to save solution calculated after each iteration
    x[0,:] = x0 # Assign 1st row as the Initial Guess
    
    for k in range(Nmax-1): # Run loop for set maximum iterations
        for i in range(len(b)): # To individually compute each element of x, can be parallelized
            x[k+1,i] = b[i]/A[i,i] # c term in x = Tx + x 
            for j in range(len(b)): # Incorporate the aij(k)*xj(k) for all j except j==i
                if j < i: # For j<i, use the updated xi 
                    x[k+1,i] =  x[k+1,i]-(A[i,j]*x[k+1,j])/A[i,i] 
                if j > i: 
                    x[k+1,i] =  x[k+1,i]-(A[i,j]*x[k,j])/A[i,i]
                    
        if max(abs(x[k+1,:] - x[k,:])) < tol: # Check if solution converges below a set tolerance using the l-infinity norm
            xfinal = x[0:k+2,:] # Save solutions for all the iterations 
            break # Exit out of the loop
        elif max(abs(x[k+1,:] - x[k,:])) > tol_div: # Check if solution diverges beyond a set tolerance, if does then stop the loop and rerun with updated arguments
            print('Solution is diverging \n')
            break # Exit out of the loop
    
    iterations = k+1
    return xfinal, iterations    

## Jacobi Method (Splitting Method)

In [30]:
# Jacobi Method:
def jacm(A,b,x0, tol, tol_div, Nmax):
    """Dhruv Jain, 4 Oct, 2021, MSAAE Purdue
    Obj: Use Jacobi Method (Splitting Method) to iteratively solve for a solution of linear system of equations of the form Ax = b, x is the solution 
    Args:
        A: (n x n) matrix, Coefficeient matrix
        b: (n x 1) vector, right-hand side vector
        x0: (n x 1) vecotr, inital guess for x*
        tol: ||x_n+1 - x_n||infi, step convergence tolerance 
        tol_div: ||x_n+1 - x_n||infi, step Divergence tolerance
        Nmax: max allowable iterations for the iterative solver
    Output:
        xfinal: (niter x n), contains solved x for each iteration, last row is the converged solution
        iterations: # iterations to converge  
    """
    x = np.zeros((Nmax,len(b))) # Create x to save solution calculated after each iteration
    x[0,:] = x0 # Assign 1st row as the Initial Guess
    
    for k in range(Nmax-1): # Run loop for set maximum iterations
        for i in range(len(b)): # To individually compute each element of x, can be parallelized
            x[k+1,i] = b[i]/A[i,i] # c term in x = Tx + x 
            for j in range(len(b)): 
                if (i != j): # Incorporate the aij(k)*xj(k) for all j except j==i
                    x[k+1,i] =  x[k+1,i]-(A[i,j]*x[k,j])/A[i,i]
        
        if max(abs(x[k+1,:] - x[k,:])) < tol: # Check if solution converges below a set tolerance using the l-infinity norm
            xfinal = x[0:k+2,:] # Save solutions for all the iterations 
            break # Exit out of the loop
        elif max(abs(x[k+1,:] - x[k,:])) > tol_div: # Check if solution diverges beyond a set tolerance, if does then stop the loop and rerun with updated arguments
            print('Solution is diverging \n')
            break # Exit out of the loop
    
    iterations = k+1
    return xfinal, iterations

## SOR Method (Splitting Method)

In [35]:
# SOR Method:
def sorm(A,b,x0, tol, tol_div, Nmax, W):
    """Dhruv Jain, 4 Oct, 2021, MSAAE Purdue
    Obj: Use SOR Method (Splitting Method) to iteratively solve for a solution of linear system of equations of the form Ax = b, x is the solution 
    Args:
        A: (n x n) matrix, Coefficeient matrix
        b: (n x 1) vector, right-hand side vector
        x0: (n x 1) vecotr, inital guess for x*
        tol: ||x_n+1 - x_n||infi, step convergence tolerance 
        tol_div: ||x_n+1 - x_n||infi, step Divergence tolerance
        Nmax: max allowable iterations for the iterative solver
        W: (n_steps x 1) vector, relaxation parameter
    Output:
        xfinal: (niter x n), contains solved x for each iteration, last row is the converged solution
        wf: Relaxation paramter list for which the solution converged
        iterations: # iterations to converge  
    """
    
    xfinal = []
    wf = []
    iterations = []
    for w in W:
        x = np.zeros((Nmax,len(b))) # Create x to save solution calculated after each iteration
        x[0,:] = x0 # Assign 1st row as the Initial Guess
        check = 0
        for k in range(Nmax-1): # Run loop for set maximum iterations
            for i in range(len(b)): # To individually compute each element of x, can be parallelized
                x[k+1,i] = w*b[i]/A[i,i] + (1-w)*x[k,i] # c term in x = Tx + x 
                for j in range(len(b)): 
                    if j < i:
                        x[k+1,i] =  x[k+1,i]-w*(A[i,j]*x[k+1,j])/A[i,i] 
                    if j > i: # Incorporate the aij(k)*xj(k) for all j except j==i
                        x[k+1,i] =  x[k+1,i]-w*(A[i,j]*x[k,j])/A[i,i]

            if max(abs(x[k+1,:] - x[k,:])) < tol: # Check if solution converges below a set tolerance using the l-infinity norm
                xfinal.append(x[0:k+1,:]) # Save solutions for all the iterations 
                wf.append(w)
                iterations.append(k+1)
                check = 1
                break # Exit out of the loop
            elif max(abs(x[k+1,:] - x[k,:])) > tol_div: # Check if solution diverges beyond a set tolerance, if does then stop the loop and rerun with updated arguments
                print('Solution is diverging for w = ',round(w,2))
                check = 1
                break # Exit out of the loop
    
        if check == 0:
            print('No solution for given convergence tolerance and w = ',round(w,2),' for set Nmax found.')
            temp = np.zeros(len(b))
            temp[:] = np.nan
            x[1,:] = temp
            xfinal.append(x[0:2,:])
            wf.append(w)
            iterations.append(k+1)
    return xfinal, wf, iterations    

## Example

In [38]:
# A and b matrix for the linear systems of equations
A = np.zeros((6,6))
for i in range(6):
    A[i,i] = -88.824
    if i < 5:
        A[i,i+1] = 48.024
    if i > 0:
        A[i,i-1] = 40.8
b = np.array([-2.04,0,0,0,0,-20.01])    
x0 = np.zeros(6)

#x0 = np.array([1,1,1,1])
tol = 10e-4
tol_div = 10e4
Nmax = 50

# Gaussian-Elimination with no Pivoting
x_a = ge_lin_equ(A,b,opt=1)
print('Solution of the system using Gaussian Elimination with No Pivoting:',x_a)

# Gaussian-Elimination with Sacled Partial Pivoting
x_b = ge_lin_equ(A,b,opt=2)
print('Solution of the system using Gaussian Elimination with Scaled Partial Pivoting:',x_b)

# Complex Gradient Method
xfinal_1, iter_1 = cgm(A,b,x0,tol,tol_div, Nmax)
print('Solution of the system using Complex Gradient Method:',xfinal_1[-1,:])

# Gauss-Sidel
xfinal_a, iterations = gsm(A,b,x0, tol, tol_div, Nmax)
print('Solution of the system using Gauss-Sidel Method:',xfinal_a[-1,:])

# Jacobi Method
xfinal_b, iterations = jacm(A,b,x0, tol, tol_div, Nmax)
print('Solution of the system using Jacobi Method:',xfinal_b[-1,:])

# SOR Method
W = [1]
xfinal_c, wf, iterations = sorm(A,b,x0, tol, tol_div, Nmax, W)
print('Solution of the system using SOR Method:',xfinal_c[0][-1,:])

Solution of the system using Gaussian Elimination with No Pivoting: [0.13104672 0.19990201 0.25839976 0.30809799 0.35032038 0.38619147]
Solution of the system using Gaussian Elimination with Scaled Partial Pivoting: [0.13104672 0.19990201 0.25839976 0.30809799 0.35032038 0.38619147]
Solution of the system using Complex Gradient Method: [0.13107739 0.19976316 0.25835912 0.30799941 0.35048153 0.38637662]
Solution of the system using Gauss-Sidel Method: [0.12867346 0.19636241 0.25474647 0.30507419 0.34831331 0.38526955]
Solution of the system using Jacobi Method: [0.12862484 0.19624009 0.25377642 0.30421854 0.34717047 0.38472467]
Solution of the system using SOR Method: [0.12810361 0.19551249 0.25386926 0.30434812 0.34783137 0.38504819]
